## Please input your directory for the top level folder

In [1]:
dir_ = 'E:/Seminararbeit/Code/A1/' # input only here

#### setting other directory

In [2]:
raw_data_dir = dir_+'2. data/'
processed_data_dir = dir_+'2. data/processed/'
log_dir = dir_+'4. logs/'
model_dir = dir_+'5. models/'
submission_dir = dir_+'6. submissions/'

In [3]:
####################################################################################
##################### 1-3. recursive model by store & dept #########################
####################################################################################

In [4]:
ver, KKK = 'priv', 1

In [5]:
STORES = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
DEPTS = ['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1', 'FOODS_2', 'FOODS_3']

In [6]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

# custom imports
from multiprocessing import Pool

warnings.filterwarnings('ignore')

In [7]:
########################### Helpers
#################################################################################
## Seeder
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [8]:
########################### Helper to load data by store ID
#################################################################################
# Read data
def get_data_by_store(store, dept):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    df = df[df['d']>=START_TRAIN]
    
    df = df[(df['store_id']==store) & (df['dept_id']==dept)]

    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2
    
    df = pd.concat([df, df3], axis=1)
    del df3
    
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    df = df.reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES:
        for state_id in DEPTS:
            temp_df = pd.read_pickle(processed_data_dir+'test_'+store_id+'_'+state_id+'.pkl')
            temp_df['store_id'] = store_id
            temp_df['dept_id'] = state_id
            base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [9]:
########################### Model params
#################################################################################
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.015,
                    'num_leaves': 2**8-1,
                    'min_data_in_leaf': 2**8-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 3000,
                    'boost_from_average': False,
                    'verbose': -1
                } 

In [10]:
########################### Vars
#################################################################################
VER = 1                          
SEED = 42                        
seed_everything(SEED)            
lgb_params['seed'] = SEED        
N_CORES = psutil.cpu_count()     


#LIMITS and const
TARGET      = 'sales'            
START_TRAIN = 700                
END_TRAIN   = 1941 - 28*KKK      
P_HORIZON   = 28                 
USE_AUX     = False             

remove_features = ['id','cat_id', 'state_id','store_id','dept_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_item_id_store_id_mean','enc_item_id_store_id_std'] 

ORIGINAL = raw_data_dir
BASE     = processed_data_dir+'processedgrid_part_1.pkl'
PRICE    = processed_data_dir+'processedgrid_part_2.pkl'
CALENDAR = processed_data_dir+'processedgrid_part_3.pkl'
LAGS     = processed_data_dir+'processedlags_df_28.pkl'
MEAN_ENC = processed_data_dir+'processedmean_encoding_df.pkl'


#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])

In [11]:
_, MODEL_FEATURES = get_data_by_store(STORES[-1], DEPTS[-1])
del _; gc.collect()

0

In [12]:
########################### Predict
#################################################################################

all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

main_time = time.time()

for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    grid_df = base_test.copy()
    
    # slow for loop version
    temp = []
    for a in ROLS_SPLIT:
        temp.append(make_lag_roll(a))
    temp = pd.concat(temp, axis=1)
    grid_df = pd.concat([grid_df, temp], axis=1)
    del temp; gc.collect()
    ###
    
    # fast multiprocessing version
#     grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)
    ###
    
    for store_id in STORES:
        for state_id in DEPTS:
        
            model_path = model_dir+'lgb_model_'+store_id+'_'+state_id+'_v'+str(VER)+'.bin'
            if USE_AUX:
                model_path = AUX_MODELS + model_path

            estimator = pickle.load(open(model_path, 'rb'))

            day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
            store_mask = base_test['store_id']==store_id
            state_mask = base_test['dept_id']==state_id

            mask = (day_mask)&(store_mask)&(state_mask)
            base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])
    
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds

Predict | Day: 1
##########  2.94 min round |  2.94 min total |  37235.62 day sales |
Predict | Day: 2
##########  2.90 min round |  5.85 min total |  35666.00 day sales |
Predict | Day: 3
##########  2.89 min round |  8.74 min total |  35111.17 day sales |
Predict | Day: 4
##########  2.86 min round |  11.60 min total |  35670.27 day sales |
Predict | Day: 5
##########  2.97 min round |  14.57 min total |  41981.09 day sales |
Predict | Day: 6
##########  2.88 min round |  17.45 min total |  51167.47 day sales |
Predict | Day: 7
##########  2.88 min round |  20.32 min total |  54277.83 day sales |
Predict | Day: 8
##########  2.87 min round |  23.20 min total |  44638.23 day sales |
Predict | Day: 9
##########  2.88 min round |  26.08 min total |  44863.96 day sales |
Predict | Day: 10
##########  2.88 min round |  28.96 min total |  39514.59 day sales |
Predict | Day: 11
##########  2.89 min round |  31.85 min total |  40801.42 day sales |
Predict | Day: 12
##########  2.87 min round

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.799927,0.772724,0.764112,0.880378,0.990771,1.741182,2.029620,0.761549,0.781596,...,0.880124,1.334448,1.095455,0.960329,0.765107,0.824674,0.929891,1.205205,1.345028,1.125643
1,HOBBIES_1_002_CA_1_evaluation,0.173316,0.178508,0.137261,0.132027,0.159516,0.188746,0.186295,0.186236,0.217918,...,0.230739,0.268079,0.265129,0.238470,0.242851,0.239941,0.234909,0.217037,0.297643,0.272131
2,HOBBIES_1_003_CA_1_evaluation,0.352776,0.368627,0.411027,0.417214,0.651729,1.206851,0.739413,0.388486,0.437813,...,0.663843,0.794842,0.864841,0.478941,0.436448,0.426103,0.475984,0.679008,0.724619,0.741344
3,HOBBIES_1_004_CA_1_evaluation,1.594861,1.338960,1.484863,1.505753,2.398626,2.010849,5.260374,1.560185,1.255845,...,1.672990,2.314300,3.010407,1.772524,1.424043,1.286733,1.294112,1.591070,3.141232,3.855611
4,HOBBIES_1_005_CA_1_evaluation,0.946563,0.994256,0.876218,1.212108,1.121769,1.196505,1.964862,1.068783,1.166742,...,1.118524,1.743332,1.411777,1.044614,0.898319,0.952915,0.873641,1.074025,1.383988,1.592838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,0.374584,0.339850,0.353514,0.372094,0.472426,0.405977,0.557681,0.559257,0.619831,...,0.605758,0.703704,0.835884,0.558241,0.497683,0.503031,0.477493,0.556647,0.634773,0.743839
30486,FOODS_3_824_WI_3_evaluation,0.301802,0.306624,0.315777,0.285400,0.299622,0.250444,0.279291,0.394560,0.432932,...,0.295035,0.343769,0.431439,0.302282,0.252271,0.249513,0.231823,0.219221,0.278736,0.294839
30487,FOODS_3_825_WI_3_evaluation,0.578341,0.473111,0.462747,0.465308,0.535376,0.907616,0.887313,1.175855,1.088633,...,1.016358,1.316070,1.553567,1.056237,0.762789,0.735355,0.635632,0.728866,0.884449,0.805866
30488,FOODS_3_826_WI_3_evaluation,1.105958,1.159991,1.002856,0.981979,1.075781,1.536217,1.064442,1.444656,1.340458,...,0.923441,1.278273,1.464129,1.177274,1.216188,0.921373,0.998096,1.032207,1.175204,1.324165


In [13]:
########################### Export
#################################################################################
submission = pd.read_csv(ORIGINAL+'sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv(submission_dir+'before_ensemble/submission_kaggle_recursive_store_dept.csv', index=False)